In [1]:
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

DEVICE  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LATENT  = 100
BATCH   = 128
EPOCHS  = 30          # stops in ~25 min on a T4
SMOOTH  = 0.9         # real-label smoothing


In [2]:
tfm = transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize((0.5,), (0.5,))])
train_ds = datasets.MNIST(root="data", train=True, download=True, transform=tfm)
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)


100%|██████████| 9.91M/9.91M [00:00<00:00, 11.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 344kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.18MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.67MB/s]


In [3]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(10, 50)
        self.fc = nn.Sequential(
            nn.Linear(LATENT + 50, 128*7*7),
            nn.BatchNorm1d(128*7*7), nn.ReLU(True))
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # 7→14
            nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1),    # 14→28
            nn.Tanh())
    def forward(self, z, y):
        x = torch.cat([z, self.embed(y)], 1)      # concat noise + label-emb
        x = self.fc(x).view(-1, 128, 7, 7)
        return self.conv(x)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(10, 28*28)
        self.conv = nn.Sequential(
            nn.Conv2d(2, 64, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            nn.Flatten(),
            nn.Linear(128*7*7, 1))
    def forward(self, x, y):
        yimg = self.embed(y).view(-1, 1, 28, 28)
        x = torch.cat([x, yimg], 1)               # image-concat label mask
        return self.conv(x)


In [4]:
G, D = Generator().to(DEVICE), Discriminator().to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
opt_G   = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
opt_D   = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))


In [5]:
for epoch in range(EPOCHS):
    for real, labels in tqdm(train_dl, leave=False):
        real, labels = real.to(DEVICE), labels.to(DEVICE)
        bs = real.size(0)

        # ---- Train Discriminator ----
        noise  = torch.randn(bs, LATENT, device=DEVICE)
        fake   = G(noise, labels)
        real_y = torch.full((bs,1), SMOOTH,  device=DEVICE)  # 0.9
        fake_y = torch.zeros(bs,1, device=DEVICE)            # 0.0

        D_real = D(real, labels)
        D_fake = D(fake.detach(), labels)
        loss_D = loss_fn(D_real, real_y) + loss_fn(D_fake, fake_y)

        opt_D.zero_grad();  loss_D.backward();  opt_D.step()

        # ---- Train Generator ----
        trick = torch.ones(bs,1, device=DEVICE)              # wants D to say 'real'
        D_fake_for_G = D(fake, labels)
        loss_G = loss_fn(D_fake_for_G, trick)

        opt_G.zero_grad();  loss_G.backward();  opt_G.step()

    print(f"Epoch {epoch+1}/{EPOCHS}  |  D: {loss_D.item():.3f}  G: {loss_G.item():.3f}")


Epoch 1/30  |  D: 0.920  G: 1.637


Epoch 2/30  |  D: 0.927  G: 0.985


Epoch 3/30  |  D: 0.927  G: 1.428


Epoch 4/30  |  D: 1.013  G: 1.946


Epoch 5/30  |  D: 1.016  G: 1.832


...

Epoch 26/30  |  D: 0.937  G: 1.667


Epoch 27/30  |  D: 1.048  G: 1.152


Epoch 28/30  |  D: 1.229  G: 2.233


Epoch 29/30  |  D: 0.924  G: 1.611


Epoch 30/30  |  D: 0.971  G: 1.526


In [6]:
torch.save(G.state_dict(), "cgan_mnist_G.pth")     # ~5 MB
